In [1]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from keras import backend as K
import argparse
import os
from sentence_embeddings import Sentence_Embedder

# Neural Network construction and testing
In this section a Neural Network is created and tested, to see which hyperparameters (for example: different activation function ReLU, hidden layer size...) have a positive / negative impact on the results. These experiments are performed with the same Datasets and concatenated vectors, as those which were used in the initial training/testing of Ridle with Natural Language Models.

In [2]:
parser = argparse.ArgumentParser(
    description='Instance Type Prediction using Ridle',
)
#parser.add_argument('--dataset', nargs='?', default='umls', type=str)
parser.add_argument('--dataset', nargs='?', default='DBp_2016-04', type=str)
parser, unknown = parser.parse_known_args()

# Load Representations
print('Reading Data...')
df = pd.read_csv('./dataset/{}/embedding.csv'.format(parser.dataset))


# Load mapping
if 'dbp' in parser.dataset.lower():
    mapping = pd.read_json('./dataset/dbp_type_mapping.json')
elif 'wd' in parser.dataset.lower() or 'wikidata' in parser.dataset.lower():
    mapping = pd.read_json('./dataset/wd_mapping_type.json')
else:
    mapping = pd.read_json('./dataset/{}/type_mapping.json'.format(parser.dataset))

# merge them
print('Processing Data...')
r = pd.merge(df, mapping, on='S')
# GELU Activation function
def gelu(x):
    return 0.5 * x * (1 + K.tanh(x * 0.7978845608 * (1 + 0.044715 * x * x)))

Reading Data...
Processing Data...


### Creation of Concatenated Vectors for Neural Network

In [3]:
print("Reading input data...")
input_data = pd.read_pickle('./dataset/{}/input_data_summary_full.pkl'.format(parser.dataset))
text = input_data['Summary']
print("Removing entities with empty summary...")
non_empty = input_data[input_data.Summary!='']
r_copy = r.copy()
r_new = r_copy[r_copy.S.isin(non_empty['S'].values)]
input_new = input_data[input_data.S.isin(non_empty['S'].values)]
inputs = r_new.drop(['S', 'Class'], axis=1).values
sentences = ["[CLS] " + temp + " [SEP]" for temp in text]
print("Embedding sentences...")
s = Sentence_Embedder
s = Sentence_Embedder(model_name="all-MiniLM-L6-v2")
print("Creating sentence embeddings...")
embeddings = s.embed(sentences)
emb_array = np.array(embeddings)
inp_array = np.array(inputs)

Reading input data...
Creating sentence embeddings...


In [4]:
print("Concatenating Vectors...")
i = 0
dict_embeddings = {}
for entity in input_new["S"]:
    dict_embeddings[entity] = [embeddings[i]]
    i+=1
    
inputs = np.array(inputs)
ridle_dict = {}
i=0
for entity in r_new['S']:
    ridle_dict[entity] = [inputs[i]]
    i+=1
    
import numpy as np
ds = [ridle_dict, dict_embeddings]
d = {}
for k in ridle_dict.keys():
    d[k] = tuple(d[k] for d in ds)
    
conc_input =  []
for key in d:
    conc_input.append(np.concatenate((d[key][0][0], d[key][1][0])))
conc_input = np.array(conc_input)

emb_array = np.array(embeddings)

Concatenating Vectors...


### Neural Network
Here we change the hyperparameters of the neural network and run it three times to get an average F1 score. The optrions used right now are the following: 
Activation function = Relu, Hidden layers = 2, Hidden layer size = input size + number of classes. For further information run the cell below...

In [33]:
Sequential?

In [6]:
print("Running Neural Network...")
K_FOLD = 10
mlb = MultiLabelBinarizer()
fold_no = 1
loss_per_fold, f1_macro, f1_micro, f1_weighted = [], [], [], []
kfold = KFold(n_splits=K_FOLD, shuffle=True, random_state=42)
targets = mlb.fit_transform(r_new['Class'])
for train, test_new in kfold.split(conc_input, targets):
    model = Sequential()
    model.add(Dense(conc_input[train].shape[1] + targets[train].shape[1], input_dim=conc_input[train].shape[1], activation="relu"))
    model.add(Dense(conc_input[train].shape[1] + targets[train].shape[1], input_dim=conc_input[train].shape[1], activation="relu"))
    model.add(Dense(targets[train].shape[1], activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('Training...')
    history = model.fit(conc_input[train], targets[train], batch_size=64, validation_data=(conc_input[test_new], targets[test_new]), epochs=100)
    y_pred = model.predict(conc_input[test_new])
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    
    # Generate F1 scores
    scores = model.evaluate(conc_input[test_new], targets[test_new], verbose=0)
    f1_macro.append(f1_score(targets[test_new], y_pred, average='macro', zero_division=1))
    f1_micro.append(f1_score(targets[test_new], y_pred, average='micro', zero_division=1))
    f1_weighted.append(f1_score(targets[test_new], y_pred, average='weighted', zero_division=1))

    print('Score for fold', fold_no, ':', model.metrics_names[0], 'of', scores[0], ';', 'F1-Macro:', f1_macro[-1], 'F1-Micro:', f1_micro[-1])
    loss_per_fold.append(scores[0])

    fold_no += 1
    
# Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_per_fold)):
    print('------------------------------------------------------------------------')
    print('> Fold', i+1, ' - Loss:', loss_per_fold[i], '- F1-Macro:', f1_macro[i], '%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print('> F1-Macro:', np.mean(f1_macro), '(+-', np.std(f1_macro), ')')
print('> F1-Micro:', np.mean(f1_micro), '(+-', np.std(f1_micro), ')')
print('> Loss:', np.mean(loss_per_fold))
print('------------------------------------------------------------------------')

# Save results to file
result = {}
f1_macro = np.array(f1_macro)
f1_micro = np.array(f1_micro)
f1_weighted = np.array(f1_weighted)
result['F1-Macro'] = np.mean(f1_macro)
result['F1-Macro_std'] = np.std(f1_macro)
result['F1-Micro'] = np.mean(f1_micro)
result['F1-Micro_std'] = np.std(f1_micro)
result['F1-Weighted'] = np.mean(f1_weighted)
result['F1-Weighted_std'] = np.std(f1_weighted)
result['Dataset'] = parser.dataset
result['method'] = 'Concatenated Vectors'
df_result = pd.DataFrame([result])
print(df_result)

model.summary()

Running Neural Network...
Training...
Epoch 1/100
690/690 [==============================] - 5s 7ms/step - loss: 0.0141 - accuracy: 0.4269 - val_loss: 0.0033 - val_accuracy: 0.4383
Epoch 2/100
690/690 [==============================] - 4s 6ms/step - loss: 0.0025 - accuracy: 0.4344 - val_loss: 0.0025 - val_accuracy: 0.5068
Epoch 3/100
690/690 [==============================] - 4s 6ms/step - loss: 0.0020 - accuracy: 0.4505 - val_loss: 0.0022 - val_accuracy: 0.4091
Epoch 4/100
690/690 [==============================] - 5s 7ms/step - loss: 0.0017 - accuracy: 0.4355 - val_loss: 0.0020 - val_accuracy: 0.4940
Epoch 5/100
690/690 [==============================] - 5s 7ms/step - loss: 0.0015 - accuracy: 0.4506 - val_loss: 0.0019 - val_accuracy: 0.4548
Epoch 6/100
690/690 [==============================] - 4s 6ms/step - loss: 0.0013 - accuracy: 0.4585 - val_loss: 0.0019 - val_accuracy: 0.4826
Epoch 7/100
690/690 [==============================] - 4s 6ms/step - loss: 0.0012 - accuracy: 0.4478 - v

Epoch 11/100
690/690 [==============================] - 5s 7ms/step - loss: 7.4481e-04 - accuracy: 0.4291 - val_loss: 0.0019 - val_accuracy: 0.3960
Epoch 12/100
690/690 [==============================] - 5s 7ms/step - loss: 6.6408e-04 - accuracy: 0.4338 - val_loss: 0.0020 - val_accuracy: 0.3989
Epoch 13/100
690/690 [==============================] - 5s 7ms/step - loss: 5.5478e-04 - accuracy: 0.4415 - val_loss: 0.0021 - val_accuracy: 0.4103
Epoch 14/100
690/690 [==============================] - 5s 7ms/step - loss: 4.9248e-04 - accuracy: 0.4691 - val_loss: 0.0021 - val_accuracy: 0.5248
Epoch 15/100
690/690 [==============================] - 5s 7ms/step - loss: 3.8848e-04 - accuracy: 0.4757 - val_loss: 0.0025 - val_accuracy: 0.4215
Epoch 16/100
690/690 [==============================] - 5s 7ms/step - loss: 3.7123e-04 - accuracy: 0.4570 - val_loss: 0.0024 - val_accuracy: 0.4552
Epoch 17/100
690/690 [==============================] - 5s 7ms/step - loss: 2.8745e-04 - accuracy: 0.4845 - val_

690/690 [==============================] - 4s 6ms/step - loss: 2.4175e-04 - accuracy: 0.4395 - val_loss: 0.0035 - val_accuracy: 0.4091
Epoch 21/100
690/690 [==============================] - 4s 6ms/step - loss: 1.6051e-04 - accuracy: 0.4244 - val_loss: 0.0037 - val_accuracy: 0.3785
Epoch 22/100
690/690 [==============================] - 4s 6ms/step - loss: 1.3525e-04 - accuracy: 0.4169 - val_loss: 0.0038 - val_accuracy: 0.3936
Epoch 23/100
690/690 [==============================] - 4s 6ms/step - loss: 1.2965e-04 - accuracy: 0.4223 - val_loss: 0.0043 - val_accuracy: 0.4279
Epoch 24/100
690/690 [==============================] - 4s 6ms/step - loss: 1.7805e-04 - accuracy: 0.3653 - val_loss: 0.0038 - val_accuracy: 0.3218
Epoch 25/100
690/690 [==============================] - 4s 6ms/step - loss: 1.3735e-04 - accuracy: 0.4068 - val_loss: 0.0042 - val_accuracy: 0.4717
Epoch 26/100
690/690 [==============================] - 4s 6ms/step - loss: 1.2318e-04 - accuracy: 0.4248 - val_loss: 0.0042 

690/690 [==============================] - 4s 6ms/step - loss: 1.2608e-04 - accuracy: 0.4743 - val_loss: 0.0033 - val_accuracy: 0.4599
Epoch 30/100
690/690 [==============================] - 4s 6ms/step - loss: 8.9361e-05 - accuracy: 0.4321 - val_loss: 0.0037 - val_accuracy: 0.4371
Epoch 31/100
690/690 [==============================] - 4s 6ms/step - loss: 1.3100e-04 - accuracy: 0.4539 - val_loss: 0.0036 - val_accuracy: 0.5713
Epoch 32/100
690/690 [==============================] - 4s 6ms/step - loss: 7.8509e-05 - accuracy: 0.4621 - val_loss: 0.0037 - val_accuracy: 0.4458
Epoch 33/100
690/690 [==============================] - 4s 6ms/step - loss: 1.3876e-04 - accuracy: 0.4417 - val_loss: 0.0037 - val_accuracy: 0.4766
Epoch 34/100
690/690 [==============================] - 4s 6ms/step - loss: 1.1861e-04 - accuracy: 0.4422 - val_loss: 0.0040 - val_accuracy: 0.4505
Epoch 35/100
690/690 [==============================] - 4s 6ms/step - loss: 8.7972e-05 - accuracy: 0.4414 - val_loss: 0.0039 

Epoch 39/100
690/690 [==============================] - 4s 6ms/step - loss: 7.0692e-05 - accuracy: 0.4727 - val_loss: 0.0046 - val_accuracy: 0.5418
Epoch 40/100
690/690 [==============================] - 4s 6ms/step - loss: 5.8866e-05 - accuracy: 0.4833 - val_loss: 0.0047 - val_accuracy: 0.4637
Epoch 41/100
690/690 [==============================] - 4s 6ms/step - loss: 1.0142e-04 - accuracy: 0.4767 - val_loss: 0.0045 - val_accuracy: 0.5435
Epoch 42/100
690/690 [==============================] - 4s 6ms/step - loss: 4.3052e-05 - accuracy: 0.4943 - val_loss: 0.0049 - val_accuracy: 0.4812
Epoch 43/100
690/690 [==============================] - 4s 6ms/step - loss: 9.3082e-05 - accuracy: 0.4789 - val_loss: 0.0049 - val_accuracy: 0.5798
Epoch 44/100
690/690 [==============================] - 4s 6ms/step - loss: 7.0054e-05 - accuracy: 0.5157 - val_loss: 0.0049 - val_accuracy: 0.4318
Epoch 45/100
690/690 [==============================] - 4s 6ms/step - loss: 7.3154e-05 - accuracy: 0.4780 - val_

...

690/690 [==============================] - 4s 6ms/step - loss: 6.7819e-05 - accuracy: 0.4450 - val_loss: 0.0053 - val_accuracy: 0.4422
Epoch 59/100
690/690 [==============================] - 4s 6ms/step - loss: 6.0420e-05 - accuracy: 0.4530 - val_loss: 0.0055 - val_accuracy: 0.4424
Epoch 60/100
690/690 [==============================] - 4s 6ms/step - loss: 6.8602e-05 - accuracy: 0.4465 - val_loss: 0.0057 - val_accuracy: 0.3953
Epoch 61/100
690/690 [==============================] - 4s 6ms/step - loss: 7.4307e-05 - accuracy: 0.4448 - val_loss: 0.0056 - val_accuracy: 0.4820
Epoch 62/100
690/690 [==============================] - 4s 6ms/step - loss: 4.6130e-05 - accuracy: 0.4547 - val_loss: 0.0058 - val_accuracy: 0.5582
Epoch 63/100
690/690 [==============================] - 4s 6ms/step - loss: 3.3744e-05 - accuracy: 0.4429 - val_loss: 0.0062 - val_accuracy: 0.3547
Epoch 64/100
690/690 [==============================] - 4s 6ms/step - loss: 8.0917e-05 - accuracy: 0.4410 - val_loss: 0.0059 

690/690 [==============================] - 4s 6ms/step - loss: 4.9798e-05 - accuracy: 0.4240 - val_loss: 0.0060 - val_accuracy: 0.4100
Epoch 68/100
690/690 [==============================] - 4s 6ms/step - loss: 4.9581e-05 - accuracy: 0.4322 - val_loss: 0.0058 - val_accuracy: 0.4455
Epoch 69/100
690/690 [==============================] - 4s 6ms/step - loss: 4.5387e-05 - accuracy: 0.4327 - val_loss: 0.0057 - val_accuracy: 0.4780
Epoch 70/100
690/690 [==============================] - 4s 6ms/step - loss: 5.8161e-05 - accuracy: 0.4864 - val_loss: 0.0058 - val_accuracy: 0.5027
Epoch 71/100
690/690 [==============================] - 4s 6ms/step - loss: 8.5544e-05 - accuracy: 0.4820 - val_loss: 0.0058 - val_accuracy: 0.5096
Epoch 72/100
690/690 [==============================] - 4s 6ms/step - loss: 4.9048e-05 - accuracy: 0.5302 - val_loss: 0.0059 - val_accuracy: 0.4688
Epoch 73/100
690/690 [==============================] - 4s 6ms/step - loss: 5.2887e-05 - accuracy: 0.5508 - val_loss: 0.0060 

Epoch 22/100
690/690 [==============================] - 4s 6ms/step - loss: 1.5577e-04 - accuracy: 0.4765 - val_loss: 0.0032 - val_accuracy: 0.4555
Epoch 23/100
690/690 [==============================] - 4s 6ms/step - loss: 1.3397e-04 - accuracy: 0.4682 - val_loss: 0.0032 - val_accuracy: 0.4090
Epoch 24/100
690/690 [==============================] - 4s 6ms/step - loss: 1.3162e-04 - accuracy: 0.4438 - val_loss: 0.0034 - val_accuracy: 0.4327
Epoch 25/100
690/690 [==============================] - 4s 6ms/step - loss: 1.2874e-04 - accuracy: 0.4358 - val_loss: 0.0033 - val_accuracy: 0.3504
Epoch 26/100
690/690 [==============================] - 4s 6ms/step - loss: 1.5235e-04 - accuracy: 0.4259 - val_loss: 0.0035 - val_accuracy: 0.4959
Epoch 27/100
690/690 [==============================] - 4s 6ms/step - loss: 1.2752e-04 - accuracy: 0.4498 - val_loss: 0.0037 - val_accuracy: 0.4214
Epoch 28/100
690/690 [==============================] - ETA: 0s - loss: 1.1048e-04 - accuracy: 0.46 - 4s 6ms/ste

690/690 [==============================] - 4s 6ms/step - loss: 3.4013e-05 - accuracy: 0.4572 - val_loss: 0.0059 - val_accuracy: 0.4745
Epoch 78/100
690/690 [==============================] - 4s 6ms/step - loss: 5.6683e-05 - accuracy: 0.4912 - val_loss: 0.0060 - val_accuracy: 0.4843
Epoch 79/100
690/690 [==============================] - 4s 6ms/step - loss: 6.1968e-05 - accuracy: 0.4824 - val_loss: 0.0057 - val_accuracy: 0.5263
Epoch 80/100
690/690 [==============================] - 4s 6ms/step - loss: 2.5683e-05 - accuracy: 0.4595 - val_loss: 0.0057 - val_accuracy: 0.4208
Epoch 81/100
690/690 [==============================] - 4s 6ms/step - loss: 4.6148e-05 - accuracy: 0.4389 - val_loss: 0.0056 - val_accuracy: 0.4506
Epoch 82/100
690/690 [==============================] - 4s 6ms/step - loss: 5.7737e-05 - accuracy: 0.4226 - val_loss: 0.0056 - val_accuracy: 0.3892
Epoch 83/100
690/690 [==============================] - 4s 6ms/step - loss: 5.0820e-05 - accuracy: 0.4730 - val_loss: 0.0059 

690/690 [==============================] - 5s 7ms/step - loss: 4.4838e-05 - accuracy: 0.4230 - val_loss: 0.0072 - val_accuracy: 0.5020
Epoch 87/100
690/690 [==============================] - 5s 7ms/step - loss: 5.9289e-05 - accuracy: 0.4904 - val_loss: 0.0075 - val_accuracy: 0.4496
Epoch 88/100
690/690 [==============================] - 5s 7ms/step - loss: 4.5914e-05 - accuracy: 0.4527 - val_loss: 0.0074 - val_accuracy: 0.4086
Epoch 89/100
690/690 [==============================] - 5s 7ms/step - loss: 3.5641e-05 - accuracy: 0.5075 - val_loss: 0.0074 - val_accuracy: 0.5071
Epoch 90/100
690/690 [==============================] - 5s 7ms/step - loss: 4.8920e-05 - accuracy: 0.5096 - val_loss: 0.0069 - val_accuracy: 0.5331
Epoch 91/100
690/690 [==============================] - 5s 7ms/step - loss: 4.9366e-05 - accuracy: 0.4966 - val_loss: 0.0073 - val_accuracy: 0.4939
Epoch 92/100
690/690 [==============================] - 5s 7ms/step - loss: 4.7254e-05 - accuracy: 0.5193 - val_loss: 0.0074 

In [44]:
print("Running Neural Network...")
K_FOLD = 10
mlb = MultiLabelBinarizer()
fold_no = 1
loss_per_fold, f1_macro, f1_micro, f1_weighted = [], [], [], []
kfold = KFold(n_splits=K_FOLD, shuffle=True, random_state=42)
targets = mlb.fit_transform(r_new['Class'])
for train, test_new in kfold.split(conc_input, targets):
    model = Sequential()
    model.add(Dense(conc_input[train].shape[1] + targets[train].shape[1], input_dim=conc_input[train].shape[1], activation="relu"))
    model.add(Dense(conc_input[train].shape[1] + targets[train].shape[1], input_dim=conc_input[train].shape[1], activation="relu"))
    model.add(Dense(targets[train].shape[1], activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('Training...')
    history = model.fit(conc_input[train], targets[train], batch_size=64, validation_data=(conc_input[test_new], targets[test_new]), epochs=100)
    y_pred = model.predict(conc_input[test_new])
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    
    # Generate F1 scores
    scores = model.evaluate(conc_input[test_new], targets[test_new], verbose=0)
    f1_macro.append(f1_score(targets[test_new], y_pred, average='macro', zero_division=1))
    f1_micro.append(f1_score(targets[test_new], y_pred, average='micro', zero_division=1))
    f1_weighted.append(f1_score(targets[test_new], y_pred, average='weighted', zero_division=1))

    print('Score for fold', fold_no, ':', model.metrics_names[0], 'of', scores[0], ';', 'F1-Macro:', f1_macro[-1], 'F1-Micro:', f1_micro[-1])
    loss_per_fold.append(scores[0])

    fold_no += 1
    
# Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_per_fold)):
    print('------------------------------------------------------------------------')
    print('> Fold', i+1, ' - Loss:', loss_per_fold[i], '- F1-Macro:', f1_macro[i], '%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print('> F1-Macro:', np.mean(f1_macro), '(+-', np.std(f1_macro), ')')
print('> F1-Micro:', np.mean(f1_micro), '(+-', np.std(f1_micro), ')')
print('> Loss:', np.mean(loss_per_fold))
print('------------------------------------------------------------------------')

# Save results to file
result = {}
f1_macro = np.array(f1_macro)
f1_micro = np.array(f1_micro)
f1_weighted = np.array(f1_weighted)
result['F1-Macro'] = np.mean(f1_macro)
result['F1-Macro_std'] = np.std(f1_macro)
result['F1-Micro'] = np.mean(f1_micro)
result['F1-Micro_std'] = np.std(f1_micro)
result['F1-Weighted'] = np.mean(f1_weighted)
result['F1-Weighted_std'] = np.std(f1_weighted)
result['Dataset'] = parser.dataset
result['method'] = 'Concatenated Vectors'
df_result = pd.DataFrame([result])
print(df_result)

model.summary()

Training...
Epoch 1/100
690/690 [==============================] - 5s 7ms/step - loss: 0.0136 - accuracy: 0.4039 - val_loss: 0.0028 - val_accuracy: 0.3671
Epoch 2/100
690/690 [==============================] - 5s 7ms/step - loss: 0.0021 - accuracy: 0.4510 - val_loss: 0.0019 - val_accuracy: 0.4405
Epoch 3/100
690/690 [==============================] - 5s 7ms/step - loss: 0.0016 - accuracy: 0.4529 - val_loss: 0.0016 - val_accuracy: 0.4258
Epoch 4/100
690/690 [==============================] - 5s 7ms/step - loss: 0.0013 - accuracy: 0.4477 - val_loss: 0.0015 - val_accuracy: 0.4983
Epoch 5/100
690/690 [==============================] - 5s 7ms/step - loss: 0.0011 - accuracy: 0.4549 - val_loss: 0.0015 - val_accuracy: 0.4452
Epoch 6/100
690/690 [==============================] - 5s 7ms/step - loss: 9.3847e-04 - accuracy: 0.4491 - val_loss: 0.0014 - val_accuracy: 0.4271
Epoch 7/100
690/690 [==============================] - 5s 7ms/step - loss: 8.1906e-04 - accuracy: 0.4693 - val_loss: 0.0014 - 

Epoch 11/100
690/690 [==============================] - 5s 7ms/step - loss: 4.4308e-04 - accuracy: 0.4232 - val_loss: 0.0013 - val_accuracy: 0.4095
Epoch 12/100
690/690 [==============================] - 5s 7ms/step - loss: 3.6851e-04 - accuracy: 0.4080 - val_loss: 0.0014 - val_accuracy: 0.4171
Epoch 13/100
690/690 [==============================] - 5s 7ms/step - loss: 3.1634e-04 - accuracy: 0.4623 - val_loss: 0.0013 - val_accuracy: 0.4473
Epoch 14/100
690/690 [==============================] - 5s 7ms/step - loss: 2.7723e-04 - accuracy: 0.4385 - val_loss: 0.0015 - val_accuracy: 0.3871
Epoch 15/100
690/690 [==============================] - 5s 7ms/step - loss: 2.1852e-04 - accuracy: 0.4180 - val_loss: 0.0015 - val_accuracy: 0.4770
Epoch 16/100
690/690 [==============================] - 5s 7ms/step - loss: 2.0217e-04 - accuracy: 0.4398 - val_loss: 0.0016 - val_accuracy: 0.3654
Epoch 17/100
690/690 [==============================] - 5s 7ms/step - loss: 1.5158e-04 - accuracy: 0.4231 - val_

Epoch 21/100
690/690 [==============================] - 5s 7ms/step - loss: 1.2110e-04 - accuracy: 0.4571 - val_loss: 0.0025 - val_accuracy: 0.4407
Epoch 22/100
690/690 [==============================] - 5s 7ms/step - loss: 8.6692e-05 - accuracy: 0.4045 - val_loss: 0.0025 - val_accuracy: 0.3620
Epoch 23/100
690/690 [==============================] - 5s 7ms/step - loss: 9.9253e-05 - accuracy: 0.3623 - val_loss: 0.0026 - val_accuracy: 0.3124
Epoch 24/100
690/690 [==============================] - 5s 7ms/step - loss: 6.4204e-05 - accuracy: 0.3995 - val_loss: 0.0027 - val_accuracy: 0.3683
Epoch 25/100
690/690 [==============================] - 5s 7ms/step - loss: 1.3157e-04 - accuracy: 0.3748 - val_loss: 0.0026 - val_accuracy: 0.4081
Epoch 26/100
690/690 [==============================] - 5s 7ms/step - loss: 6.0520e-05 - accuracy: 0.3943 - val_loss: 0.0029 - val_accuracy: 0.3932
Epoch 27/100
690/690 [==============================] - 5s 7ms/step - loss: 5.2482e-05 - accuracy: 0.4276 - val_

Epoch 31/100
690/690 [==============================] - 5s 7ms/step - loss: 6.7864e-05 - accuracy: 0.3801 - val_loss: 0.0023 - val_accuracy: 0.4058
Epoch 32/100
690/690 [==============================] - 5s 7ms/step - loss: 6.6118e-05 - accuracy: 0.4015 - val_loss: 0.0023 - val_accuracy: 0.4111
Epoch 33/100
690/690 [==============================] - 5s 7ms/step - loss: 5.5446e-05 - accuracy: 0.3880 - val_loss: 0.0022 - val_accuracy: 0.3989
Epoch 34/100
690/690 [==============================] - 5s 7ms/step - loss: 7.0621e-05 - accuracy: 0.4343 - val_loss: 0.0023 - val_accuracy: 0.4034
Epoch 35/100
690/690 [==============================] - 5s 7ms/step - loss: 5.7435e-05 - accuracy: 0.3901 - val_loss: 0.0024 - val_accuracy: 0.3991
Epoch 36/100
690/690 [==============================] - 5s 7ms/step - loss: 3.7078e-05 - accuracy: 0.4013 - val_loss: 0.0024 - val_accuracy: 0.4481
Epoch 37/100
690/690 [==============================] - 5s 7ms/step - loss: 1.1207e-04 - accuracy: 0.3624 - val_

Epoch 41/100
690/690 [==============================] - 5s 7ms/step - loss: 6.9162e-05 - accuracy: 0.4174 - val_loss: 0.0023 - val_accuracy: 0.4076
Epoch 42/100
690/690 [==============================] - 5s 7ms/step - loss: 3.8654e-05 - accuracy: 0.4483 - val_loss: 0.0023 - val_accuracy: 0.4573
Epoch 43/100
690/690 [==============================] - 5s 7ms/step - loss: 4.6436e-05 - accuracy: 0.4745 - val_loss: 0.0027 - val_accuracy: 0.4131
Epoch 44/100
690/690 [==============================] - 5s 7ms/step - loss: 4.7185e-05 - accuracy: 0.4094 - val_loss: 0.0024 - val_accuracy: 0.4147
Epoch 45/100
690/690 [==============================] - 5s 7ms/step - loss: 4.8703e-05 - accuracy: 0.4267 - val_loss: 0.0025 - val_accuracy: 0.4343
Epoch 46/100
690/690 [==============================] - 5s 7ms/step - loss: 2.6630e-05 - accuracy: 0.4392 - val_loss: 0.0028 - val_accuracy: 0.5327
Epoch 47/100
690/690 [==============================] - 5s 7ms/step - loss: 6.7783e-05 - accuracy: 0.4551 - val_

...

690/690 [==============================] - 5s 7ms/step - loss: 2.8687e-05 - accuracy: 0.5384 - val_loss: 0.0031 - val_accuracy: 0.5122
Epoch 61/100
690/690 [==============================] - 5s 7ms/step - loss: 2.8813e-05 - accuracy: 0.4892 - val_loss: 0.0030 - val_accuracy: 0.5233
Epoch 62/100
690/690 [==============================] - 5s 7ms/step - loss: 3.6581e-05 - accuracy: 0.4972 - val_loss: 0.0034 - val_accuracy: 0.4461
Epoch 63/100
690/690 [==============================] - 5s 7ms/step - loss: 3.5346e-05 - accuracy: 0.4678 - val_loss: 0.0031 - val_accuracy: 0.4529
Epoch 64/100
690/690 [==============================] - 5s 7ms/step - loss: 5.9010e-05 - accuracy: 0.4775 - val_loss: 0.0033 - val_accuracy: 0.4196
Epoch 65/100
690/690 [==============================] - 5s 7ms/step - loss: 6.5999e-05 - accuracy: 0.4760 - val_loss: 0.0029 - val_accuracy: 0.5194
Epoch 66/100
690/690 [==============================] - 5s 7ms/step - loss: 3.8352e-05 - accuracy: 0.4837 - val_loss: 0.0031 

Epoch 15/100
690/690 [==============================] - 5s 7ms/step - loss: 2.2489e-04 - accuracy: 0.4613 - val_loss: 0.0016 - val_accuracy: 0.4718
Epoch 16/100
690/690 [==============================] - 5s 7ms/step - loss: 2.0458e-04 - accuracy: 0.4638 - val_loss: 0.0018 - val_accuracy: 0.5224
Epoch 17/100
690/690 [==============================] - 5s 7ms/step - loss: 1.5679e-04 - accuracy: 0.4666 - val_loss: 0.0019 - val_accuracy: 0.3982
Epoch 18/100
690/690 [==============================] - 5s 7ms/step - loss: 1.6975e-04 - accuracy: 0.4451 - val_loss: 0.0019 - val_accuracy: 0.4814
Epoch 19/100
690/690 [==============================] - 5s 7ms/step - loss: 1.3616e-04 - accuracy: 0.5035 - val_loss: 0.0019 - val_accuracy: 0.5624
Epoch 20/100
690/690 [==============================] - 5s 7ms/step - loss: 1.3424e-04 - accuracy: 0.4758 - val_loss: 0.0019 - val_accuracy: 0.4592
Epoch 21/100
690/690 [==============================] - 5s 7ms/step - loss: 1.0503e-04 - accuracy: 0.4306 - val_

Epoch 25/100
690/690 [==============================] - 4s 6ms/step - loss: 8.9124e-05 - accuracy: 0.4446 - val_loss: 0.0023 - val_accuracy: 0.3641
Epoch 26/100
690/690 [==============================] - 4s 6ms/step - loss: 1.0523e-04 - accuracy: 0.4560 - val_loss: 0.0022 - val_accuracy: 0.5304
Epoch 27/100
690/690 [==============================] - 4s 6ms/step - loss: 4.3636e-05 - accuracy: 0.4257 - val_loss: 0.0023 - val_accuracy: 0.3853
Epoch 28/100
690/690 [==============================] - 4s 6ms/step - loss: 5.9822e-05 - accuracy: 0.3978 - val_loss: 0.0024 - val_accuracy: 0.3553
Epoch 29/100
690/690 [==============================] - 4s 6ms/step - loss: 1.0830e-04 - accuracy: 0.4221 - val_loss: 0.0022 - val_accuracy: 0.3733
Epoch 30/100
690/690 [==============================] - 4s 6ms/step - loss: 5.3766e-05 - accuracy: 0.4015 - val_loss: 0.0023 - val_accuracy: 0.3790
Epoch 31/100
690/690 [==============================] - 4s 6ms/step - loss: 9.0450e-05 - accuracy: 0.4120 - val_

690/690 [==============================] - 4s 6ms/step - loss: 2.0202e-05 - accuracy: 0.4226 - val_loss: 0.0035 - val_accuracy: 0.4708
Epoch 81/100
690/690 [==============================] - 4s 6ms/step - loss: 1.7237e-05 - accuracy: 0.4690 - val_loss: 0.0037 - val_accuracy: 0.4290
Epoch 82/100
690/690 [==============================] - 4s 6ms/step - loss: 4.0502e-05 - accuracy: 0.4169 - val_loss: 0.0034 - val_accuracy: 0.3492
Epoch 83/100
690/690 [==============================] - 4s 6ms/step - loss: 3.6211e-05 - accuracy: 0.4124 - val_loss: 0.0038 - val_accuracy: 0.4371
Epoch 84/100
690/690 [==============================] - 4s 6ms/step - loss: 4.8774e-05 - accuracy: 0.4335 - val_loss: 0.0034 - val_accuracy: 0.3769
Epoch 85/100
690/690 [==============================] - 4s 6ms/step - loss: 9.1128e-06 - accuracy: 0.4225 - val_loss: 0.0036 - val_accuracy: 0.4263
Epoch 86/100
690/690 [==============================] - 4s 6ms/step - loss: 7.7022e-06 - accuracy: 0.4181 - val_loss: 0.0038 

Epoch 35/100
690/690 [==============================] - 4s 6ms/step - loss: 5.6584e-05 - accuracy: 0.4408 - val_loss: 0.0027 - val_accuracy: 0.4406
Epoch 36/100
690/690 [==============================] - 4s 6ms/step - loss: 2.1700e-05 - accuracy: 0.3961 - val_loss: 0.0028 - val_accuracy: 0.4333
Epoch 37/100
690/690 [==============================] - 4s 6ms/step - loss: 8.1980e-05 - accuracy: 0.4213 - val_loss: 0.0030 - val_accuracy: 0.4555
Epoch 38/100
690/690 [==============================] - 4s 6ms/step - loss: 8.3762e-05 - accuracy: 0.4982 - val_loss: 0.0028 - val_accuracy: 0.4686
Epoch 39/100
690/690 [==============================] - 4s 6ms/step - loss: 2.7110e-05 - accuracy: 0.4371 - val_loss: 0.0030 - val_accuracy: 0.4241
Epoch 40/100
690/690 [==============================] - 4s 6ms/step - loss: 8.1041e-05 - accuracy: 0.4250 - val_loss: 0.0029 - val_accuracy: 0.5894
Epoch 41/100
690/690 [==============================] - 4s 6ms/step - loss: 3.0559e-05 - accuracy: 0.5044 - val_

In [43]:
print("Running Neural Network...")
K_FOLD = 10
mlb = MultiLabelBinarizer()
fold_no = 1
loss_per_fold, f1_macro, f1_micro, f1_weighted = [], [], [], []
kfold = KFold(n_splits=K_FOLD, shuffle=True, random_state=42)
targets = mlb.fit_transform(r_new['Class'])
for train, test_new in kfold.split(conc_input, targets):
    model = Sequential()
    model.add(Dense(conc_input[train].shape[1] + targets[train].shape[1], input_dim=conc_input[train].shape[1], activation="relu"))
    model.add(Dense(conc_input[train].shape[1] + targets[train].shape[1], input_dim=conc_input[train].shape[1], activation="relu"))
    model.add(Dense(targets[train].shape[1], activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('Training...')
    history = model.fit(conc_input[train], targets[train], batch_size=64, validation_data=(conc_input[test_new], targets[test_new]), epochs=100)
    y_pred = model.predict(conc_input[test_new])
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    
    # Generate F1 scores
    scores = model.evaluate(conc_input[test_new], targets[test_new], verbose=0)
    f1_macro.append(f1_score(targets[test_new], y_pred, average='macro', zero_division=1))
    f1_micro.append(f1_score(targets[test_new], y_pred, average='micro', zero_division=1))
    f1_weighted.append(f1_score(targets[test_new], y_pred, average='weighted', zero_division=1))

    print('Score for fold', fold_no, ':', model.metrics_names[0], 'of', scores[0], ';', 'F1-Macro:', f1_macro[-1], 'F1-Micro:', f1_micro[-1])
    loss_per_fold.append(scores[0])

    fold_no += 1
    
# Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_per_fold)):
    print('------------------------------------------------------------------------')
    print('> Fold', i+1, ' - Loss:', loss_per_fold[i], '- F1-Macro:', f1_macro[i], '%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print('> F1-Macro:', np.mean(f1_macro), '(+-', np.std(f1_macro), ')')
print('> F1-Micro:', np.mean(f1_micro), '(+-', np.std(f1_micro), ')')
print('> Loss:', np.mean(loss_per_fold))
print('------------------------------------------------------------------------')

# Save results to file
result = {}
f1_macro = np.array(f1_macro)
f1_micro = np.array(f1_micro)
f1_weighted = np.array(f1_weighted)
result['F1-Macro'] = np.mean(f1_macro)
result['F1-Macro_std'] = np.std(f1_macro)
result['F1-Micro'] = np.mean(f1_micro)
result['F1-Micro_std'] = np.std(f1_micro)
result['F1-Weighted'] = np.mean(f1_weighted)
result['F1-Weighted_std'] = np.std(f1_weighted)
result['Dataset'] = parser.dataset
result['method'] = 'Concatenated Vectors'
df_result = pd.DataFrame([result])
print(df_result)

model.summary()

Training...
Epoch 1/100
690/690 [==============================] - 6s 8ms/step - loss: 0.0133 - accuracy: 0.4177 - val_loss: 0.0027 - val_accuracy: 0.3875
Epoch 2/100
690/690 [==============================] - 6s 8ms/step - loss: 0.0020 - accuracy: 0.4434 - val_loss: 0.0019 - val_accuracy: 0.5107
Epoch 3/100
690/690 [==============================] - 6s 8ms/step - loss: 0.0015 - accuracy: 0.4630 - val_loss: 0.0016 - val_accuracy: 0.4966
Epoch 4/100
690/690 [==============================] - 6s 8ms/step - loss: 0.0013 - accuracy: 0.4509 - val_loss: 0.0016 - val_accuracy: 0.5362
Epoch 5/100
690/690 [==============================] - 6s 8ms/step - loss: 0.0011 - accuracy: 0.4506 - val_loss: 0.0015 - val_accuracy: 0.4632
Epoch 6/100
690/690 [==============================] - 6s 8ms/step - loss: 9.2205e-04 - accuracy: 0.4304 - val_loss: 0.0014 - val_accuracy: 0.5603
Epoch 7/100
690/690 [==============================] - 6s 8ms/step - loss: 7.9402e-04 - accuracy: 0.4469 - val_loss: 0.0015 - 

Epoch 11/100
690/690 [==============================] - 6s 8ms/step - loss: 4.2273e-04 - accuracy: 0.4699 - val_loss: 0.0014 - val_accuracy: 0.4795
Epoch 12/100
690/690 [==============================] - 6s 8ms/step - loss: 3.6148e-04 - accuracy: 0.4669 - val_loss: 0.0014 - val_accuracy: 0.5430
Epoch 13/100
690/690 [==============================] - 6s 8ms/step - loss: 3.0397e-04 - accuracy: 0.4561 - val_loss: 0.0015 - val_accuracy: 0.4285
Epoch 14/100
690/690 [==============================] - 6s 8ms/step - loss: 2.5336e-04 - accuracy: 0.4344 - val_loss: 0.0016 - val_accuracy: 0.4456
Epoch 15/100
690/690 [==============================] - 6s 8ms/step - loss: 2.2992e-04 - accuracy: 0.4363 - val_loss: 0.0016 - val_accuracy: 0.4595
Epoch 16/100
690/690 [==============================] - 6s 8ms/step - loss: 1.8601e-04 - accuracy: 0.4315 - val_loss: 0.0018 - val_accuracy: 0.4923
Epoch 17/100
690/690 [==============================] - 6s 8ms/step - loss: 1.7218e-04 - accuracy: 0.4636 - val_

690/690 [==============================] - 5s 8ms/step - loss: 1.0513e-04 - accuracy: 0.4063 - val_loss: 0.0027 - val_accuracy: 0.4240
Epoch 21/100
690/690 [==============================] - 5s 8ms/step - loss: 1.2161e-04 - accuracy: 0.4051 - val_loss: 0.0026 - val_accuracy: 0.3267
Epoch 22/100
690/690 [==============================] - 5s 8ms/step - loss: 1.0325e-04 - accuracy: 0.3890 - val_loss: 0.0028 - val_accuracy: 0.4156
Epoch 23/100
690/690 [==============================] - 5s 8ms/step - loss: 1.0993e-04 - accuracy: 0.4210 - val_loss: 0.0025 - val_accuracy: 0.3367
Epoch 24/100
690/690 [==============================] - 6s 8ms/step - loss: 6.8059e-05 - accuracy: 0.3998 - val_loss: 0.0026 - val_accuracy: 0.4024
Epoch 25/100
690/690 [==============================] - 5s 8ms/step - loss: 1.1162e-04 - accuracy: 0.3834 - val_loss: 0.0027 - val_accuracy: 0.4317
Epoch 26/100
690/690 [==============================] - 6s 8ms/step - loss: 5.2439e-05 - accuracy: 0.3957 - val_loss: 0.0030 

Epoch 30/100
690/690 [==============================] - 6s 8ms/step - loss: 4.3681e-05 - accuracy: 0.4210 - val_loss: 0.0021 - val_accuracy: 0.4203
Epoch 31/100
690/690 [==============================] - 5s 8ms/step - loss: 6.2798e-05 - accuracy: 0.4185 - val_loss: 0.0022 - val_accuracy: 0.3783
Epoch 32/100
690/690 [==============================] - 5s 8ms/step - loss: 7.2261e-05 - accuracy: 0.3894 - val_loss: 0.0022 - val_accuracy: 0.3899
Epoch 33/100
690/690 [==============================] - 6s 8ms/step - loss: 4.3773e-05 - accuracy: 0.3912 - val_loss: 0.0025 - val_accuracy: 0.4781
Epoch 34/100
690/690 [==============================] - 6s 8ms/step - loss: 8.4143e-05 - accuracy: 0.3787 - val_loss: 0.0023 - val_accuracy: 0.4515
Epoch 35/100
690/690 [==============================] - 5s 8ms/step - loss: 6.4473e-05 - accuracy: 0.4326 - val_loss: 0.0023 - val_accuracy: 0.4464
Epoch 36/100
690/690 [==============================] - 5s 8ms/step - loss: 8.9827e-05 - accuracy: 0.4240 - val_

690/690 [==============================] - 5s 8ms/step - loss: 7.0921e-05 - accuracy: 0.4465 - val_loss: 0.0026 - val_accuracy: 0.3906
Epoch 40/100
690/690 [==============================] - 5s 8ms/step - loss: 4.4662e-05 - accuracy: 0.4281 - val_loss: 0.0024 - val_accuracy: 0.4276
Epoch 41/100
690/690 [==============================] - 5s 8ms/step - loss: 1.0493e-05 - accuracy: 0.3935 - val_loss: 0.0025 - val_accuracy: 0.4014
Epoch 42/100
690/690 [==============================] - 5s 8ms/step - loss: 2.5827e-05 - accuracy: 0.3833 - val_loss: 0.0027 - val_accuracy: 0.4286
Epoch 43/100
690/690 [==============================] - 6s 8ms/step - loss: 1.3904e-04 - accuracy: 0.3932 - val_loss: 0.0024 - val_accuracy: 0.4265
Epoch 44/100
690/690 [==============================] - 5s 8ms/step - loss: 2.6548e-05 - accuracy: 0.4272 - val_loss: 0.0025 - val_accuracy: 0.4935
Epoch 45/100
690/690 [==============================] - 5s 8ms/step - loss: 1.9759e-05 - accuracy: 0.3891 - val_loss: 0.0026 

Epoch 49/100
690/690 [==============================] - 5s 8ms/step - loss: 3.5895e-05 - accuracy: 0.4072 - val_loss: 0.0031 - val_accuracy: 0.3955
Epoch 50/100
690/690 [==============================] - 5s 8ms/step - loss: 2.0467e-05 - accuracy: 0.4282 - val_loss: 0.0032 - val_accuracy: 0.3976
Epoch 51/100
690/690 [==============================] - 6s 8ms/step - loss: 4.0833e-05 - accuracy: 0.4080 - val_loss: 0.0033 - val_accuracy: 0.3459
Epoch 52/100
690/690 [==============================] - 6s 8ms/step - loss: 9.1752e-05 - accuracy: 0.4374 - val_loss: 0.0034 - val_accuracy: 0.4612
Epoch 53/100
690/690 [==============================] - 5s 8ms/step - loss: 1.9093e-05 - accuracy: 0.4396 - val_loss: 0.0033 - val_accuracy: 0.3808
Epoch 54/100
690/690 [==============================] - 6s 8ms/step - loss: 8.2735e-06 - accuracy: 0.3816 - val_loss: 0.0033 - val_accuracy: 0.3822
Epoch 55/100
690/690 [==============================] - 6s 8ms/step - loss: 3.9849e-05 - accuracy: 0.3884 - val_

690/690 [==============================] - 5s 7ms/step - loss: 5.3715e-05 - accuracy: 0.4667 - val_loss: 0.0028 - val_accuracy: 0.5259
Epoch 59/100
690/690 [==============================] - 5s 7ms/step - loss: 3.1196e-05 - accuracy: 0.5290 - val_loss: 0.0031 - val_accuracy: 0.4935
Epoch 60/100
690/690 [==============================] - 5s 7ms/step - loss: 2.2751e-05 - accuracy: 0.4956 - val_loss: 0.0034 - val_accuracy: 0.4676
Epoch 61/100
690/690 [==============================] - 5s 7ms/step - loss: 6.7841e-05 - accuracy: 0.4915 - val_loss: 0.0028 - val_accuracy: 0.4478
Epoch 62/100
690/690 [==============================] - 5s 7ms/step - loss: 3.2860e-05 - accuracy: 0.4588 - val_loss: 0.0031 - val_accuracy: 0.4729
Epoch 63/100
690/690 [==============================] - 5s 7ms/step - loss: 3.6404e-05 - accuracy: 0.4758 - val_loss: 0.0030 - val_accuracy: 0.4712
Epoch 64/100
690/690 [==============================] - 5s 7ms/step - loss: 1.5910e-05 - accuracy: 0.4971 - val_loss: 0.0031 

Epoch 13/100
690/690 [==============================] - 5s 7ms/step - loss: 3.0334e-04 - accuracy: 0.4448 - val_loss: 0.0015 - val_accuracy: 0.4653
Epoch 14/100
690/690 [==============================] - 5s 7ms/step - loss: 2.7164e-04 - accuracy: 0.4354 - val_loss: 0.0017 - val_accuracy: 0.4645
Epoch 15/100
690/690 [==============================] - 5s 7ms/step - loss: 2.1751e-04 - accuracy: 0.4484 - val_loss: 0.0018 - val_accuracy: 0.4243
Epoch 16/100
690/690 [==============================] - 5s 7ms/step - loss: 1.9080e-04 - accuracy: 0.4493 - val_loss: 0.0017 - val_accuracy: 0.4369
Epoch 17/100
690/690 [==============================] - 5s 7ms/step - loss: 1.8118e-04 - accuracy: 0.4542 - val_loss: 0.0018 - val_accuracy: 0.3784
Epoch 18/100
690/690 [==============================] - 5s 7ms/step - loss: 1.3371e-04 - accuracy: 0.4167 - val_loss: 0.0019 - val_accuracy: 0.3845
Epoch 19/100
690/690 [==============================] - 5s 7ms/step - loss: 1.6155e-04 - accuracy: 0.4352 - val_

Epoch 23/100
690/690 [==============================] - 5s 7ms/step - loss: 1.0210e-04 - accuracy: 0.4817 - val_loss: 0.0019 - val_accuracy: 0.4631
Epoch 24/100
690/690 [==============================] - 5s 7ms/step - loss: 9.2762e-05 - accuracy: 0.4404 - val_loss: 0.0021 - val_accuracy: 0.5224
Epoch 25/100
690/690 [==============================] - 5s 7ms/step - loss: 7.3906e-05 - accuracy: 0.4920 - val_loss: 0.0020 - val_accuracy: 0.4673
Epoch 26/100
690/690 [==============================] - 5s 7ms/step - loss: 8.2628e-05 - accuracy: 0.4613 - val_loss: 0.0021 - val_accuracy: 0.4316
Epoch 27/100
690/690 [==============================] - 5s 7ms/step - loss: 6.8885e-05 - accuracy: 0.5029 - val_loss: 0.0022 - val_accuracy: 0.5269
Epoch 28/100
690/690 [==============================] - 5s 7ms/step - loss: 8.3006e-05 - accuracy: 0.4607 - val_loss: 0.0023 - val_accuracy: 0.4384
Epoch 29/100
690/690 [==============================] - 5s 7ms/step - loss: 9.7710e-05 - accuracy: 0.4431 - val_

Epoch 33/100
690/690 [==============================] - 5s 7ms/step - loss: 1.0178e-04 - accuracy: 0.4255 - val_loss: 0.0026 - val_accuracy: 0.5080
Epoch 34/100
690/690 [==============================] - 5s 7ms/step - loss: 2.7653e-05 - accuracy: 0.4689 - val_loss: 0.0029 - val_accuracy: 0.4957
Epoch 35/100
690/690 [==============================] - 5s 7ms/step - loss: 6.4557e-05 - accuracy: 0.4466 - val_loss: 0.0029 - val_accuracy: 0.4712
Epoch 36/100
690/690 [==============================] - 5s 7ms/step - loss: 7.6987e-05 - accuracy: 0.4340 - val_loss: 0.0028 - val_accuracy: 0.4300
Epoch 37/100
690/690 [==============================] - 5s 7ms/step - loss: 4.5944e-05 - accuracy: 0.4359 - val_loss: 0.0031 - val_accuracy: 0.3788
Epoch 38/100
690/690 [==============================] - 5s 7ms/step - loss: 6.0957e-05 - accuracy: 0.4116 - val_loss: 0.0028 - val_accuracy: 0.4937
Epoch 39/100
690/690 [==============================] - 5s 7ms/step - loss: 5.5341e-05 - accuracy: 0.4605 - val_